# Crossword grid detection
- Aim is to detect crossword grid structure.

Failures
- Big bang approach to finding grid lines, had to break this into multiple steps

# Detect grids

In [2]:
import os
import json
import numpy as np
from glob import glob
from lambda_function.grid_detect import get_crossword_grid_array
import cv2


SOLUTIONS_DIR = "../dataset/solutions"
IMAGES_DIR = "../dataset/images"
MAX_FILES = 100  # set this to None to process all


def load_solution_data(json_path):
    """Load the solution JSON and return (grid, numbers)."""
    with open(json_path, "r") as f:
        data = json.load(f)
    grid = np.array(data["grid"], dtype=int)
    numbers = np.array(data.get("numbers", np.zeros_like(grid)), dtype=int)
    return grid, numbers


def compare_grids(grid1, grid2):
    """Compare two grids and return True if identical."""
    if grid1.shape != grid2.shape:
        return False
    return np.array_equal(grid1, grid2)


def main(max_files=None):
    solution_files = glob(os.path.join(SOLUTIONS_DIR, "*.json"))

    if max_files is not None:
        solution_files = solution_files[:max_files]

    total = 0
    matching_both = 0
    matching_grid_only = 0
    non_matching = 0

    for sol_path in solution_files:
        base = os.path.basename(sol_path).replace("_solution.json", "")
        img_path = os.path.join(IMAGES_DIR, f"{base}.png")

        if not os.path.exists(img_path):
            print(f"⚠️ No image found for {sol_path}")
            continue

        try:
            sol_grid, sol_numbers = load_solution_data(sol_path)
            image = cv2.imread(str(img_path))
            if image is None:
                raise FileNotFoundError(f"Could not load image {img_path}")

            # Unpack detected results
            det_grid, det_numbers, across_clues, down_clues = get_crossword_grid_array(image)

            grid_match = compare_grids(sol_grid, det_grid)
            num_match = compare_grids(sol_numbers, det_numbers)

            if grid_match and num_match:
                print(f"✅ Full match: {base}")
                matching_both += 1
            elif grid_match and not num_match:
                print(f"⚠️ Grid matches but numbers differ: {base}")
                matching_grid_only += 1
            else:
                print(f"❌ Grid mismatch: {base} "
                      f"(solution {sol_grid.shape}, detected {det_grid.shape})")
                non_matching += 1

            total += 1

        except Exception as e:
            print(f"⚠️ Error processing {base}: {e}")
            non_matching += 1
            total += 1

    print("\n==== SUMMARY ====")
    print(f"Total puzzles:          {total}")
    print(f"Matching grid+numbers: {matching_both}")
    print(f"Matching grid only:    {matching_grid_only}")
    print(f"Non-matching:          {non_matching}")


if __name__ == "__main__":
    main(max_files=MAX_FILES)


Error: Could not load image from dataset/images/daily-1994-02-Feb0494.png


[ WARN:0@0.056] global loadsave.cpp:275 findDecoder imread_('dataset/images/daily-1994-02-Feb0494.png'): can't open/read file: check file path/integrity


✅ Full match: daily-1999-09-Sep1399
✅ Full match: daily-2015-07-Jul0715
✅ Full match: daily-1996-09-Sep1096
✅ Full match: daily-2014-09-Sep1514
✅ Full match: daily-2005-10-Oct1805
✅ Full match: daily-2002-04-Apr0702
✅ Full match: daily-2004-12-Dec0204
✅ Full match: daily-1994-01-Jan2194
✅ Full match: daily-1997-12-Dec1897
✅ Full match: daily-1998-01-Jan0698
❌ Grid mismatch: daily-1994-11-Nov2094 (solution (21, 21), detected (14, 21))
✅ Full match: daily-2009-07-Jul1009
✅ Full match: daily-2009-12-Dec1709
✅ Full match: daily-2015-02-Feb1115
✅ Full match: daily-2009-01-Jan1009
✅ Full match: bonus-2015-Jul15_sp
✅ Full match: daily-2010-10-Oct2510
✅ Full match: daily-2001-06-Jun1801
✅ Full match: daily-2011-07-Jul2111
✅ Full match: daily-2006-09-Sep0706
✅ Full match: daily-2006-11-Nov2106
✅ Full match: daily-2000-11-Nov3000
✅ Full match: daily-2019-01-Jan0919
✅ Full match: daily-1997-10-Oct2597
✅ Full match: daily-2005-12-Dec0105
✅ Full match: daily-2014-08-Aug2014
✅ Full match: daily-2015